In [2]:
import httplib2
from bs4 import BeautifulSoup, SoupStrainer
import pandas as pd
import requests
import logging
import sys


http = httplib2.Http()

depth_const = 5
visited_links=[]
subcategory=""



In [3]:
def get_contents(link, node):
    #category = 'Anger'
    #link = 'https://vaniquotes.org/wiki/Angry_means'
    page = requests.get(link)
    cid =0
    soup = BeautifulSoup(page.content, "html.parser")
    content_type = soup.find_all("div", {"class": "section"})
    if not content_type: content_type = soup.find_all("div", {"id": "section"})
    all_contents = soup.find_all("div", {"class": "quote"})

    
    for i,content in enumerate(all_contents):
        if len(content_type)==1:
            ref=0
        else: ref=i
        c_type=content_type[ref].find_all(class_="mw-headline")[0].get('id')
        reference = content.get("id") if content.get("id") else content.find_all(class_="quote_link")[0].text

        temp_str=''
        for para in content.find_all("p"):
            temp_str = temp_str + para.get_text()+'\n'
        cid=cid+1
        #list_of_contents.append([c_type, reference, temp_str])
    

def my_custom_logger(logger_name, level=logging.INFO):
    """
    Method to return a custom logger with the given name and level
    """
    logger = logging.getLogger(logger_name)
    logger.setLevel(level) 
    file_handler = logging.FileHandler(logger_name, mode='a')
    logger.addHandler(file_handler)
    return logger


In [4]:


def recursiveUrl(url,subcategory,depth):
    
    if depth>depth_const:
        return
    
    start=False
    no_of_links=1
    status, response = http.request(url)
    link_list=[]
    for index, link in enumerate(BeautifulSoup(response,'html.parser', parse_only=SoupStrainer('a'))):
        #try:
            #print(link)
            if link.has_attr('href'):
                #print(link['href'])
                if  "https://vaniquotes.org/w/index.php?" in link['href'] and "from" not in link['href'] :
                    start =False
                    
                if '/wiki/Special:Categories' in link['href']:
                    break
                    
                if start and link['href'].startswith("/wiki/")  and link['href'] not in visited_links :
                    #print(url+" Depth - "+ str(depth)+" Link: "+link['href']+" "+str(no_of_links))
                    visited_links.append(link['href'])
                    if '/Category:' not in link['href']:
                        logger.info("subcategory=%s,link=%s",subcategory,"https://vaniquotes.org"+link["href"])
                        #logger.info("level1=%s","https://vaniquotes.org"+link['href'])
                        #print(url+" Depth - "+ str(depth)+" Link: "+link['href']+" "+str(no_of_links))
                    else:
                        recursiveUrl("https://vaniquotes.org"+link['href'],subcategory,depth + 1)
                            
                            
                if "https://vanimedia.org/wiki/Main_Page" in link['href']:
                     start =True            

        #except:
            #pass

start=False
#categories = ["God","krsna",
categories = ["Yoga","Time"]
#categories = ["Duties,_Principles,_Rules,_and_Responsibilities","Living_Entity_(Jiva)"]


for category in categories:
    url = 'https://vaniquotes.org/wiki/Category:'+category+'_-_Umbrella_Category'
    #url = "https://vaniquotes.org/wiki/Category:God_-_Umbrella_Category"

    file_name = './'+category+'_subcategory.log'
    logger = my_custom_logger(file_name)
    status, response = http.request(url)
    
    
    
    for index, link in enumerate(BeautifulSoup(response, 'html.parser', parse_only=SoupStrainer('a'))):
        #try:
            if 'html' not in link and link.has_attr('href'):
                #print(link['href'])
                if  "https://vaniquotes.org/w/index.php?" in link['href'] and "from" not in link['href']:
                    start =False
                if '/wiki/Special:Categories' in link['href']:
                    break
                if start and link['href'].startswith("/wiki/") and link['href'] not in visited_links:
                    visited_links.append(link['href'])
                    subcategory=link['href']
                    
                    recursiveUrl("https://vaniquotes.org"+link['href'],subcategory, 0) 

                if "https://vanimedia.org/wiki/Main_Page" in link['href']:
                    start =True            

        #except Exception:
            #logger.exception("Fatal error in main loop")
        

In [40]:
len(visited_links)

99

In [11]:


def recursiveUrl(url,depth):
    
    if depth>depth_const:
        return
    
    start=False
    no_of_links=1
    status, response = http.request(url)
    link_list=[]
    for index, link in enumerate(BeautifulSoup(response,'html.parser', parse_only=SoupStrainer('a'))):
        #try:
            #print(link)
            if link.has_attr('href'):
                #print(link['href'])
                if  "https://vaniquotes.org/w/index.php?" in link['href'] and "from" not in link['href'] :
                    start =False
                if '/wiki/Special:Categories' in link['href']:
                    break
                if start and link['href'].startswith("/wiki/")  and link['href'] not in visited_links :
                    print(url+" Depth - "+ str(depth)+" Link: "+link['href'])
                    visited_links.append(link['href'])
                    if '/Category:' not in link['href']:
                        link_list.append([url,link['href']])
                    else:
                        recursiveUrl("https://vaniquotes.org"+link['href'],depth + 1)
                            
                            
                if "https://vanimedia.org/wiki/Main_Page" in link['href']:
                     start =True            

        #except:
            #pass

#url = 'https://vaniquotes.org/wiki/Category:Adherent_Devotees_of_God_Are_Assured_All_Protection_from_God'
url = "https://vaniquotes.org/wiki/Category:Adhere"

file_name = './'+'adhere'+'_subcategory.log'
logger = my_custom_logger(file_name)
status, response = http.request(url)

visited_links=[]

link_list=[]

start =False

for index, link in enumerate(BeautifulSoup(response, 'html.parser', parse_only=SoupStrainer('a'))):
    #try:
        if 'html' not in link and link.has_attr('href'):
            #print(link['href'])
            if  "https://vaniquotes.org/w/index.php?" in link['href'] and "from" not in link['href']:
                start =False
            if '/wiki/Special:Categories' in link['href']:
                break
                
            if start and link['href'].startswith("/wiki/") and link['href'] not in visited_links:
                visited_links.append(link['href'])
                link_list.append([url,link['href']])
                recursiveUrl("https://vaniquotes.org"+link['href'], 0) 

            if "https://vanimedia.org/wiki/Main_Page" in link['href']:
                start =True            

https://vaniquotes.org/wiki/Category:Adherence_of_a_Devotee_of_God Depth - 0 Link: /wiki/Category:Real_Devotees_of_God_Adhere_to_the_Regulative_Principles_of_Devotional_Service_Strictly
https://vaniquotes.org/wiki/Category:Real_Devotees_of_God_Adhere_to_the_Regulative_Principles_of_Devotional_Service_Strictly Depth - 1 Link: /wiki/Devotee_always_thinks_of_his_special_devotional_service_%26_attachment._Such_attachment_does_not_violate_the_regulative_principles_of_devotional_service,_%26_a_real_devotee_adheres_to_these_principles_strictly;_yet_always_thinks_of_his_particular_attachment
https://vaniquotes.org/wiki/Category:Adherence_of_a_Devotee_of_God Depth - 0 Link: /wiki/If_we_adhere_to_the_order_of_Sri_Caitanya_Mahaprabhu,_we_shall_get_spiritual_strength_without_a_doubt,_and_we_shall_be_free_to_preach_this_cult_of_the_Hare_Krsna_movement_and_not_be_hampered_by_anyone
https://vaniquotes.org/wiki/Category:Adherence_of_a_Devotee_of_God Depth - 0 Link: /wiki/They_(Jagannatha_Misra_%26_Sac

https://vaniquotes.org/wiki/Category:Kanistha-adhikari_Devotees_of_God_Remain_Stuck_Up_in_Deity_Worship Depth - 1 Link: /wiki/Simply_we_remain_stuck_up_in_Deity_worship,_we_do_not_feel_for_others_--_na_canyesu_na_tad-bhakta_--_you_do_not_know_who_is_devotee,_how_to_worship_him,_then_we_remain_kanistha-adhikari
https://vaniquotes.org/wiki/Category:Stick_To Depth - 0 Link: /wiki/Category:Sticking_to_Devotional_Service_to_God
https://vaniquotes.org/wiki/Category:Sticking_to_Devotional_Service_to_God Depth - 1 Link: /wiki/If_one_is_very_serious_about_liberation,_he_must_stick_to_the_process_of_transcendental_loving_service,_engaging_twenty-four_hours_a_day_in_the_highest_stage_of_ecstasy,_and_he_must_certainly_be_aloof_from_all_activities_of_sense_gratification
https://vaniquotes.org/wiki/Category:Sticking_to_Devotional_Service_to_God Depth - 1 Link: /wiki/Krsna_said_to_Uddhava,_%22If_one_sticks_to_devotional_service_and_is_conducted_by_regulative_principles_given_by_the_authorities_and_ac

https://vaniquotes.org/wiki/Category:Strongly_Adhere_to_God Depth - 0 Link: /wiki/In_SB_3.25.38_the_Lord_says:_Persons_who_are_satisfied_with_the_idea_of_becoming_My_adherents_alone._They_consider_that_I_am_their_soul,_their_friend,_I_am_their_son,_I_am_their_master,_I_am_their_well-wisher,_I_am_their_God,_%26_I_am_their_supreme_goal
https://vaniquotes.org/wiki/Category:Strongly_Adhere_to_God Depth - 0 Link: /wiki/In_Srimad-Bhagavatam_(SB_3.25.38)_the_Lord_says:_Persons_who_are_satisfied_with_the_idea_of_becoming_My_adherents_alone._They_consider_that_I_am_their_soul,_I_am_their_God,_and_I_am_their_supreme_goal._My_dear_mother,_time_does_not_act_on_such_devotees
https://vaniquotes.org/wiki/Category:Strongly_Adhere_to_God Depth - 0 Link: /wiki/In_Srimad-Bhagavatam_(SB_3.25.38)_the_Lord_says:_The_word_mat-para_is_only_used_to_refer_to_persons_who_are_satisfied_with_the_idea_of_becoming_My_adherents_alone
https://vaniquotes.org/wiki/Category:Strongly_Adhere_to_God Depth - 0 Link: /wiki/Th

In [10]:
link_list2=link_list

In [39]:
df=pd.DataFrame(link_list, columns=["cat","Link"])
df.to_excel("asdhere_data.xlsx")

In [38]:

response = requests.get("https://vaniquotes.org/wiki/Category:Adhere")

with open('response2.txt','w', encoding="utf-8") as f:
    f.write(link_list)

TypeError: write() argument must be str, not list

In [16]:
visited_links

['/wiki/Category:Adherence_of_a_Devotee_of_God',
 '/wiki/Category:Real_Devotees_of_God_Adhere_to_the_Regulative_Principles_of_Devotional_Service_Strictly',
 '/wiki/Devotee_always_thinks_of_his_special_devotional_service_%26_attachment._Such_attachment_does_not_violate_the_regulative_principles_of_devotional_service,_%26_a_real_devotee_adheres_to_these_principles_strictly;_yet_always_thinks_of_his_particular_attachment',
 '/wiki/If_we_adhere_to_the_order_of_Sri_Caitanya_Mahaprabhu,_we_shall_get_spiritual_strength_without_a_doubt,_and_we_shall_be_free_to_preach_this_cult_of_the_Hare_Krsna_movement_and_not_be_hampered_by_anyone',
 '/wiki/They_(Jagannatha_Misra_%26_Sacimata)_knew_that_it_was_by_the_grace_of_the_Lord_that_they_were_endowed_with_such_happiness_and_opulence._Therefore_instead_of_forgetting_the_Lord,_they_became_more_%26_more_adherent_in_rendering_service_to_the_lotus_feet_of_God',
 '/wiki/Category:Adherent_Devotees_of_God_Are_Assured_All_Protection_from_God',
 '/wiki/A_devote